# T027 · Kinase similarity: Ligand profile

**Note:** This talktorial is a part of TeachOpenCADD, a platform that aims to teach domain-specific skills and to provide pipeline templates as starting points for research projects.

Authors:

- Talia B. Kimber, 2021, [Volkamer lab, Charité](https://volkamerlab.org/)
- Dominique Sydow, 2021, [Volkamer lab, Charité](https://volkamerlab.org/)
- Andrea Volkamer, 2021, [Volkamer lab, Charité](https://volkamerlab.org/)

## Aim of this talktorial

The aim of this talktorial is to investigate kinase similarity through ligand profiling data (ChEMBL29). In the context of drug design, the following assumption is often made: if a compound was tested active on two different kinases, it is suspected that these two kinases may have some degree of similarity. We will use this assumption in this talktorial. The concept of kinase promiscuity is also covered.

### Contents in *Theory*

* Kinase dataset
* Bioactivity data
* Kinase similarity descriptor: Ligand profile
    * Kinase similarity
    * Kinase promiscuity
* From similarity matrix to distance matrix

### Contents in *Practical*

* Define the kinases of interest
* Retrieve the data
* Preprocess the data
    * Hit or non-hit
* Kinase promiscuity
* Kinase similarity
    * Visualize similarity as kinase matrix
    * Save kinase similarity matrix
* Kinase distance matrix
    * Save kinase distance matrix

### References

* Kinase dataset: [<i>Molecules</i> (2021), <b>26(3)</b>, 629](https://www.mdpi.com/1420-3049/26/3/629) 
* ChEMBL database
  * Website: https://www.ebi.ac.uk/chembl/
  * Paper: [<i>Nucleic Acid Research</i> (2017), <b>45(D1)</b>, D945-D954](https://doi.org/10.1093/nar/gkw1074) 
* KinMap
  * Website: http://www.kinhub.org/kinmap/
  * Paper: [<i>BMC Bioinformatics</i> (2017), <b>18(1)</b>, 16](https://doi.org/10.1186/s12859-016-1433-7) 

## Theory

### Kinase dataset

We use the kinase selection as defined in __Talktorial T023__.

### Bioactivity data

In order to measure kinase similarity through ligand profiling data, bioactivity data is retrieved from the well-known [ChEMBL](https://www.ebi.ac.uk/chembl/) database and the query focuses on human kinases. Luckily, a curated version of ChEMBL29 is already freely available through the Openkinome organization, see https://github.com/openkinome/kinodata.
For more details on querying the ChEMBL database, please refer to __Talktorial T001__.

In drug design, it is common to binarize the activity of a compound against a target of interest as a "hit" or "non-hit". Practically speaking, this is done using a cutoff value for measured activity. If the activity is greater than the cutoff, the compound is labeled as active (hit), and inactive (non-hit) otherwise.

![Manning tree with number of ChEMBL activities per kinase (KinMap)](images/kinmap_n_activities_per_kinase.png)


*Figure 1:* 
Number of ChEMBL29 bioactivities per kinase - as collected in [kinodata](https://github.com/openkinome/kinodata) - mapped onto the Manning kinome tree using [KinMap](http://www.kinhub.org/kinmap/). The figure has been generated in __Talktorial T023__.

### Kinase similarity descriptor: Ligand profile

As a measure of similarity, we use ligand profiling data in this talktorial.

#### Kinase similarity

We use the following metric as similarity between two kinases $K_i$ and $K_j$ :

$$
\text{similarity}(K_i, K_j) = \frac{\#\text{ of compounds that were tested active on both } K_i \text{ and } K_j}
{\#\text{ of compounds that were tested on both } K_i \text{ and } K_j}.
$$

Assuming that only one compound was tested on two kinases, and that the compound was tested as active for one and inactive for the other, then the similarity between these two kinases would be zero.

#### Kinase promiscuity
Computing the similarity between a kinase and itself may be interpreted as kinase promiscuity, where the similarity described above would therefore represent the fraction of active compounds over all tested compounds.

### From similarity matrix to distance matrix

As discussed in __Talktorial T024__, we convert the similarity matrix to a distance matrix.

## Practical

In [1]:
from pathlib import Path
from collections import Counter

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

In [2]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

In [3]:
configs = pd.read_csv(HERE / "../T023_what_is_a_kinase/data/pipeline_configs.csv")
configs = configs.set_index("variable")["default_value"]
DEMO = bool(int(configs["DEMO"]))
print(f"Run in demo mode: {DEMO}")
# NBVAL_CHECK_OUTPUT

Run in demo mode: True


### Define the kinases of interest

Let's load the kinase selection as defined in __Talktorial T023__.

In [4]:
kinase_selection_df = pd.read_csv(
    HERE
    / "../T023_what_is_a_kinase/\
data/kinase_selection.csv"
)
kinase_selection_df
# NBVAL_CHECK_OUTPUT

,kinase,kinase_klifs,uniprot_id,group,full_kinase_name
0,EGFR,EGFR,P00533,TK,Epidermal growth factor receptor
1,ErbB2,ErbB2,P04626,TK,Erythroblastic leukemia viral oncogene homolog 2
2,PI3K,p110a,P42336,Atypical,Phosphatidylinositol-3-kinase
3,VEGFR2,KDR,P35968,TK,Vascular endothelial growth factor receptor 2
4,BRAF,BRAF,P15056,TKL,Rapidly accelerated fibrosarcoma isoform B
5,CDK2,CDK2,P24941,CMGC,Cyclic-dependent kinase 2
6,LCK,LCK,P06239,TK,Lymphocyte-specific protein tyrosine kinase
7,MET,MET,P08581,TK,Mesenchymal-epithelial transition factor
8,p38a,p38a,Q16539,CMGC,p38 mitogen activated protein kinase alpha


### Retrieve the data

We retrieve a pre-curated version of a kinase subset of ChEMBL29 freely available at Openkinome, see https://github.com/openkinome/kinodata/releases/tag/v0.3.

In [ ]:
path = "https://github.com/openkinome/kinodata/releases/download/\
v0.3/activities-chembl29_v0.3.zip"
# Load data and reset index so that it starts from 0
data = pd.read_csv(path, index_col=0).reset_index(drop=True)
print(f"Current shape of data: {data.shape}")
data.head()
# NBVAL_CHECK_OUTPUT

### Preprocess the data

We look at the type of activity and the associated units.

In [ ]:
print(
    f"Activities: {sorted(set(data['activities.standard_type']))}\n"
    f"Units: {set(data['activities.standard_units'])}"
)

Let's keep the entries which have pIC50 values only.

In [ ]:
data = data[data["activities.standard_type"] == "pIC50"]

In [ ]:
data.columns
# NBVAL_CHECK_OUTPUT

The DataFrame contains many columns that won't be necessary for the rest of the notebook which are therefore removed. 
Only relevant information is kept, namely the canonical SMILES of the compound (`compound_structures.canonical_smiles`), the measured activity (`activities.standard_value`) and the UniProt ID of the kinase (`UniprotID`). These columns are renamed for readability.

In [ ]:
data = data[["compound_structures.canonical_smiles", "activities.standard_value", "UniprotID"]]
data = data.rename(
    columns={
        "compound_structures.canonical_smiles": "smiles",
        "activities.standard_value": "pIC50",
    }
)

In [ ]:
print(f"Current shape of data: {data.shape}")
data.head()
# NBVAL_CHECK_OUTPUT

NA values are dropped.

In [ ]:
data = data.dropna()
print(f"Current shape of data: {data.shape}")
# NBVAL_CHECK_OUTPUT

We only keep the data for the query kinases:

In [ ]:
data = data[data["UniprotID"].isin(kinase_selection_df["uniprot_id"])]
print(f"Current shape of data: {data.shape}")
data.head()
# NBVAL_CHECK_OUTPUT

Let's look at example data (which corresponds to the first row in the kinase selection DataFrame):

In [ ]:
example_kinase = kinase_selection_df["kinase_klifs"][0]
example_uniprot = kinase_selection_df["uniprot_id"][0]

example_data = data[data["UniprotID"] == example_uniprot]

print(f"Example kinase: {example_kinase}")

Some compounds have been tested several times against a target, as shown below.

In [ ]:
measured_compounds = Counter(example_data["smiles"])
try:
    top_measured_compounds = measured_compounds.most_common()[0:5]
except IndexError:
    top_measured_compounds = measured_compounds.most_common()
top_measured_compounds
# NBVAL_CHECK_OUTPUT

We have a look at those compounds.

In [ ]:
mols = []
for entry in top_measured_compounds:
    mols.append(Chem.MolFromSmiles(entry[0]))
Draw.MolsToGridImage(mols, molsPerRow=5)

In this example (demo mode), the first molecule is gefitinib, a known FDA-approved drug against EGFR.

As a simple workaround — since we prefer to have one activity value per compound-kinase pair — we keep the value for which the compound has the best activity value, i.e., the highest pIC50 value.

In [ ]:
data = data.groupby(["UniprotID", "smiles"])["pIC50"].max().reset_index()
data.head()
# NBVAL_CHECK_OUTPUT

#### Hit or non-hit

Finally, we binarize the pIC50 values to obtain hit or non-hit using a cutoff. We use a pIC50 cutoff of $6.3$, similarly to the cutoff used in [<i>Molecules</i> (2021), <b>26(3)</b>, 629](https://www.mdpi.com/1420-3049/26/3/629).

In [ ]:
cutoff = 6.3

In [ ]:
def binarize_pic50(pic50_value, threshold):
    """
    Binarizes a scalar value given a threshold.

    Parameters
    ----------
    pic50_value : float
        The measurement pIC50 value of a kinase-ligand pair.
    threshold : float
        The cutoff to determine activity.

    Returns
    -------
    int
        1 if the pIC50 value is above the threshold, which indicates activity.
        0 otherwise.
    """
    if pic50_value >= threshold:
        return 1
    else:
        return 0

In [ ]:
data["activity_binary"] = data["pIC50"].apply(binarize_pic50, args=(cutoff,))

In [ ]:
print(f"Current shape of data: {data.shape}")
data.head()
# NBVAL_CHECK_OUTPUT

### Kinase promiscuity

We now look at the kinase promiscuity.

For a given kinase, three values are computed:

1. the total number of measured compounds against the given kinase,
2. the number of active compounds against the kinase, and
3. the fraction of active compounds, i.e., the ratio of active compounds over the total number of measured compounds per kinase.

In [ ]:
def kinase_to_activity_numbers(uniprot_id, activity_df):
    """
    Retrieve the three values for a given kinase.

    Parameters
    ----------
    uniprot_id : str
        The UniProt ID of the kinase of interest, e.g. "P00533" for "EGFR".
    activity_df : pd.DataFrame
        The dataframe with activity values for kinases.

    Returns
    -------
    tuple : (int, int, float)
        The three metrics:
        1. The total number of measured compounds against the kinase.
        2. The number of active compounds against the kinase.
        3. The fraction of active compounds against the kinase.
    """
    kinase_data = activity_df[activity_df["UniprotID"] == uniprot_id]
    total_measured_compounds = len(kinase_data)
    active_compounds = len(kinase_data[kinase_data["activity_binary"] == 1])
    if total_measured_compounds > 0:
        fraction = active_compounds / total_measured_compounds
    else:
        print("No compounds were measured for this kinase.")
        fraction = np.nan
    return (total_measured_compounds, active_compounds, fraction)

Let's see what information we get for the first kinase in our dataset:

In [ ]:
example_kinase = kinase_selection_df["kinase_klifs"][0]
example_uniprot = kinase_selection_df["uniprot_id"][0]

print(f"{example_kinase} ({example_uniprot}):")
example_metrics = kinase_to_activity_numbers(example_uniprot, data)
print(
    f"{'Total number of measured compounds:' : <40}"
    f"{example_metrics[0]} \n"
    f"{'Number of active compounds:' : <40}"
    f"{example_metrics[1]} \n"
    f"{'Fraction of active compounds:' : <40}"
    f"{example_metrics[2]:.2f} \n"
)
# NBVAL_CHECK_OUTPUT

Let's create a table from these values for all kinases:

In [ ]:
def promiscuity_table(kinase_selection, activity_df):
    """
    Create a table with all three values for all kinases.

    Parameters
    ----------
    kinase_selection : pd.DataFrame
        The DataFrame for the chosen kinases.
    activity_df : pd.DataFrame
        The DataFrame with activity values for kinases.

    Returns
    -------
    promiscuity_table : pd.DataFrame
        A DataFrame with the kinases as rows and values as columns.
    """
    promiscuity_table = pd.DataFrame(
        index=kinase_selection["kinase_klifs"], columns=["total", "actives", "fraction"]
    )
    promiscuity_table.index.name = None
    promiscuity_table.columns.name = None

    for name, uniprot_id in zip(kinase_selection["kinase_klifs"], kinase_selection["uniprot_id"]):
        values = kinase_to_activity_numbers(uniprot_id, activity_df)
        promiscuity_table.loc[name] = values
    return promiscuity_table

In [ ]:
kinase_promiscuity_df = promiscuity_table(kinase_selection_df, data)
kinase_promiscuity_df
# NBVAL_CHECK_OUTPUT

Let's beautify the table:

In [ ]:
kinase_promiscuity_df.style.format("{:.3f}", subset=["fraction"]).background_gradient(
    cmap="Purples", subset=["fraction"]
).highlight_min(color="yellow", axis=None, subset=["fraction"]).highlight_max(
    color="red", subset=["fraction"]
)

From the table, we notice that CDK2 is the least (in yellow) and BRAF the most (in red) promiscuous kinase.

### Kinase similarity

We now investigate how we can use the similarity measure discussed in the _Theory_ part to compare kinases. 

In [ ]:
def similarity_ligand_profile(uniprot_id1, uniprot_id2, activity_df):
    """
    Compute the similarity between two kinases using ligand profiling data.

    Parameters
    ----------
    uniprot_id1 : str
        UniProt ID of first kinase of interest.
    uniprot_id2 : str
        UniProt ID of second kinase of interest.
    activity_df :  pd.DataFrame
        The DataFrame with activity values for kinases.

    Returns
    -------
    tuple : (int, int, float)
        The three metrics:
        1. The total number of measured compounds against both kinases.
        2. The number of active compounds against both kinases.
        3. The metric for kinase similarity,
                i.e. number of active compounds on both kinases
                over number of measured compounds on both kinases.
    """
    if uniprot_id1 == uniprot_id2:
        return kinase_to_activity_numbers(uniprot_id1, activity_df)
    else:
        # Data for the two kinases only
        reduced_data = activity_df[activity_df["UniprotID"].isin([uniprot_id1, uniprot_id2])]

        # Look at active compounds only
        active_entries = reduced_data[reduced_data["activity_binary"] == 1]
        # Group by compounds
        compounds = active_entries.groupby("smiles").size()
        # Look at the number of active compounds measured on both kinases
        active_compounds_on_both = compounds[compounds == 2].shape[0]

        # Look at all tested compounds
        compounds = reduced_data.groupby("smiles").size()
        # Look at the number of compounds measured on both kinases
        measured_compounds_on_both = compounds[compounds == 2].shape[0]

        if measured_compounds_on_both > 0:
            fraction = active_compounds_on_both / measured_compounds_on_both
        else:
            print(
                f"No compounds were measured on both kinases, "
                f"namely {uniprot_id1} and {uniprot_id2}."
            )
            fraction = np.nan
            measured_compounds_on_both = np.nan
            active_compounds_on_both = np.nan
        return (measured_compounds_on_both, active_compounds_on_both, fraction)

Let's look at the values and similarity between two kinases.

In [ ]:
if DEMO:
    kinase1 = "EGFR"
    uniprot1 = "P00533"
    kinase2 = "MET"
    uniprot2 = "P08581"
else:
    kinase1 = kinase_selection_df["kinase_klifs"][0]
    uniprot1 = kinase_selection_df["uniprot_id"][0]
    kinase2 = kinase_selection_df["kinase_klifs"][1]
    uniprot2 = kinase_selection_df["uniprot_id"][1]

similarity_example = similarity_ligand_profile(uniprot1, uniprot2, data)
print(
    f"Values for {kinase1} and {kinase2}: \n\n"
    f"{'Total number of measured compounds:' : <50}"
    f"{similarity_example[0]} \n"
    f"{'Number of active compounds:' : <50}"
    f"{similarity_example[1]} \n"
    f"Fraction of active compounds or \n"
    f"{'ligand profile similarity:' : <50}"
    f"{similarity_example[2]:.2f} \n"
)
# NBVAL_CHECK_OUTPUT

#### Visualize similarity as kinase matrix

Let's first look at the non-reduced fraction of number of active compound against total number of compounds to have an idea of the counts.

In [ ]:
kinase_counts_matrix = pd.DataFrame(
    index=kinase_selection_df.kinase_klifs, columns=kinase_selection_df.kinase_klifs
)
kinase_counts_matrix.index.name = None
kinase_counts_matrix.columns.name = None

for i, (uniprot_id1, klifs_name1) in enumerate(
    zip(kinase_selection_df.uniprot_id, kinase_selection_df.kinase_klifs)
):
    for j, (uniprot_id2, klifs_name2) in enumerate(
        zip(kinase_selection_df.uniprot_id, kinase_selection_df.kinase_klifs)
    ):
        total, actives, _ = similarity_ligand_profile(uniprot_id1, uniprot_id2, data)
        integer_ratio = f"{actives}/{total}"
        kinase_counts_matrix[klifs_name1][klifs_name2] = integer_ratio
kinase_counts_matrix
# NBVAL_CHECK_OUTPUT

Note that the total number of tested compounds as well as the number of active compounds on two kinases vary largely.

* For the p110a-ErbB2 pair, there are none.
* For p110a and [BRAF, CDK2, LCK, MET and p38a], there are less than $13$ commonly tested compounds.
* In contrast, the EGFR-ErbB2 pair has $1170$ commonly tested compounds, of which $662$ were active on both.

Now let's look at the similarity, in this case, the reduced fraction:

In [ ]:
kinase_similarity_matrix = np.zeros((len(kinase_selection_df), len(kinase_selection_df)))
for i, uniprot_id1 in enumerate(kinase_selection_df.uniprot_id):
    for j, uniprot_id2 in enumerate(kinase_selection_df.uniprot_id):
        kinase_similarity_matrix[i, j] = similarity_ligand_profile(uniprot_id1, uniprot_id2, data)[
            2
        ]

In [ ]:
kinase_similarity_matrix_df = pd.DataFrame(
    data=kinase_similarity_matrix,
    index=kinase_selection_df.kinase_klifs,
    columns=kinase_selection_df.kinase_klifs,
)
kinase_similarity_matrix_df.index.name = None
kinase_similarity_matrix_df.columns.name = None
kinase_similarity_matrix_df
# NBVAL_CHECK_OUTPUT

In [ ]:
# Show matrix with background gradient
cm = sns.light_palette("green", as_cmap=True)
kinase_similarity_matrix_df.style.background_gradient(cmap=cm).format("{:.3f}")

Note that the diagonal contains the previously discussed promiscuity values.

As mentioned above, no compounds were measured on both ErbB2 and p110a and therefore creates a `np.nan` entry which can be problematic for algorithmic reason.

As a simple workaround, we will fill the NA values with zero.

In [ ]:
kinase_similarity_matrix_df = kinase_similarity_matrix_df.fillna(0)

kinase_similarity_matrix_df.style.background_gradient(cmap=cm).format("{:.3f}")

#### Save kinase similarity matrix

In [ ]:
kinase_similarity_matrix_df.to_csv(DATA / "kinase_similarity_matrix.csv")

### Kinase distance matrix

The similarity matrix $SM$ is converted to a pseudo-distance matrix (all entries of the similarity matrix are between $0$ and $1$):

In [ ]:
print(
    f"The values of the similarity matrix lie between: "
    f"{kinase_similarity_matrix_df.min().min():.2f}"
    f" and {kinase_similarity_matrix_df.max().max():.2f}"
)
# NBVAL_CHECK_OUTPUT

In [ ]:
kinase_distance_matrix_df = 1 - kinase_similarity_matrix_df

Finally, we set the diagonal values to $0$ and we obtain the kinase distance matrix:

In [ ]:
np.fill_diagonal(kinase_distance_matrix_df.values, 0)

In [ ]:
kinase_distance_matrix_df.style.background_gradient(cmap=cm).format("{:.3f}")

#### Save kinase distance matrix

In [ ]:
kinase_distance_matrix_df.to_csv(DATA / "kinase_distance_matrix.csv")

## Discussion

In this talktorial, we investigate how activity data can be used as a measure of similarity between kinases. The fraction of compounds tested as actives over the total number of measured compounds is a way of accessing the similarity. Moreover, using the same rationale, the promiscuity of a kinase can be quantified using the ratio of active compounds over measured compounds.

When working with these data, we have to keep in mind that some kinases have much higher coverage with respect to the number of compounds that were tested against them, leading to an imbalance in information content.
This cannot be inferred from the calculated fraction. For example, the pairs EGFR-KDR and EGFR-p38a both have a profile similarity of $0.35$. However, the first was calculated based on $893$ tested compounds, whereas the latter on $52$ only.

The kinase distance matrix above will be reloaded in __Talktorial T028__, where we compare kinase similarities from different perspectives, including the ligand profile perspective we have talked about in this talktorial.

## Quiz

1. Is there an optimal way to deal with multiple kinase-ligand measurements?
2. Can promiscuity be fairly compared between two kinases if one has been tested against many compounds whereas the other only against very few? 
3. Using the similarity described in this talktorial, what does it mean that two kinases have a similarity of $0$, as is the case for p110a and LCK?